# Ergodicity Insurance - Tail Simulation - Parallel

In [1]:
!pip install --user --upgrade --force-reinstall git+https://github.com/AlexFiliakov/Ergodic-Insurance-Limits

  Cloning https://github.com/AlexFiliakov/Ergodic-Insurance-Limits to /tmp/pip-req-build-aourks7r
  Running command git clone --filter=blob:none --quiet https://github.com/AlexFiliakov/Ergodic-Insurance-Limits /tmp/pip-req-build-aourks7r
  Resolved https://github.com/AlexFiliakov/Ergodic-Insurance-Limits to commit 79b04b279455e4952047b4935f23fb6ebc2ca9a7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached numpy-2.3.3-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached pandas-2.3.3-cp312-cp312-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (91 kB)
  Using cached pydantic-2.12.0-py3-none-any.whl.metadata (83 kB)
  Using cached pyyaml-6.0.3-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (2.4 kB)
  Using cached matplotlib-3.10.7-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (11 kB

In [2]:
import sys
import os
import glob

# Add user site-packages to the system path
user_site_packages = os.path.expanduser("~/.local/lib/python*/site-packages")
sys.path.extend(glob.glob(user_site_packages)) # Use glob to handle potential version differences

import ergodic_insurance as ei

ei.__version__

'0.3.0'

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import sys # Import sys
import glob # Import glob

# Define the directory containing the module from Google Drive
module_dir = "/content/drive/MyDrive/Colab Notebooks"

# Add the directory to sys.path if it's not already there
if module_dir not in sys.path:
    sys.path.append(module_dir)
    print(f"Added {module_dir} to sys.path within cell WfwYS382ns8Y")

# Now import the module from the file in Google Drive
try:
    from run_tail_simulation_colab import run_tail_simulation
    print("Successfully imported run_basic_simulation from run_tail_simulation_colab.py within cell WfwYS382ns8Y")
except ModuleNotFoundError:
    print("Could not import run_basic_simulation from run_tail_simulation_colab.py within cell WfwYS382ns8Y.")
    print("Please ensure that run_tail_simulation_colab.py exists directly in the /content/drive directory and contains the 'run_basic_simulation' function.")
    # Re-raise the error so the user sees it
    raise
except ImportError as e:
    print(f"An error occurred during import within cell WfwYS382ns8Y: {e}")
    # Re-raise the error so the user sees it
    raise

Added /content/drive/MyDrive/Colab Notebooks to sys.path within cell WfwYS382ns8Y
Successfully imported run_basic_simulation from run_tail_simulation_colab.py within cell WfwYS382ns8Y


In [5]:
# import os
# import glob

# # Get the current working directory
# current_directory = os.getcwd()
# print(f"Current working directory: {current_directory}")

# # List all .pkl files in the current directory
# pkl_files = glob.glob(os.path.join(current_directory, '*.pkl'))

# # Delete each file
# if pkl_files:
#     for file_path in pkl_files:
#         os.remove(file_path)
#         print(f"Deleted: {file_path}")
#     print("All .pkl files in the current working directory have been deleted.")
# else:
#     print("No .pkl files found in the current working directory.")

In [6]:
from tqdm.notebook import tqdm  # progress bar
from itertools import product
import os
from joblib import Parallel, delayed
import shutil # Import shutil for file operations

from run_tail_simulation_colab import run_tail_simulation

# Integrate joblib with tqdm for per-task progress
class TqdmJoblib(tqdm):
    def __enter__(self):
        from joblib import parallel
        self._original = parallel.BatchCompletionCallBack
        outer = self
        class TqdmBatchCompletionCallback(parallel.BatchCompletionCallBack):
            def __call__(self, *args, **kwargs):
                outer.update(n=self.batch_size)
                return super().__call__(*args, **kwargs)
        parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback
        return super().__enter__()
    def __exit__(self, exc_type, exc, tb):
        from joblib import parallel
        parallel.BatchCompletionCallBack = self._original
        return super().__exit__(exc_type, exc, tb)

NUM_SIMULATIONS = 250_000
SIM_YEARS = 50
PRICING_SIMULATIONS = 500_000  # For premium estimation

initial_assets_list = [25_000_000, 50_000_000]
atr_list = [1.0]
ebitabl_list = [0.1]
deductible_list = [100_000]
loss_ratio_list = [0.5, 0.6, 0.7]
loss_ratio_list = [0.6]
policy_limit_list = [25_000_000, 50_000_000, 75_000_000, 100_000_000, 150_000_000, 250_000_000, 350_000_000, 500_000_000]
extreme_threshold_percentile = [0.001, 0.0005, 0.0001]
# extreme_threshold_percentile = [0.0005]
extreme_sev_shape = [1.0, 1.5, 2.0, 2.5]
# extreme_sev_shape = [2.5]
extreme_sev_scale_percentile = [0.5, 1.0, 1.5, 2.0]  # Scale at the 1/10th of the threshold percentile of catastrophic severity
extreme_sev_scale_percentile = [1.0]

# # Test overrides
# loss_ratio_list = [0.7]
# policy_limit_list = [50_000_000]
# extreme_threshold_percentile = [0.0005]
# extreme_sev_shape = [1.5]
# extreme_sev_scale_percentile = [1.5]  # Scale at the 1/10th of the threshold percentile of catastrophic severity

# Use all but one CPU core
_cpu = os.cpu_count() or 1
n_jobs = max(_cpu, 1)

print(f'Available cores: {n_jobs}')

# Define the destination directory in your Google Drive
# Make sure 'My Drive/results' exists in your Google Drive
gdrive_results_dir = '/content/drive/My Drive/Colab Notebooks/results'
os.makedirs(gdrive_results_dir, exist_ok=True) # Create the directory if it doesn't exist

prod_with_extreme_thresholds = product(
                                    initial_assets_list,
                                    atr_list,
                                    ebitabl_list,
                                    deductible_list,
                                    loss_ratio_list,
                                    policy_limit_list,
                                    extreme_threshold_percentile,
                                    extreme_sev_shape,
                                    extreme_sev_scale_percentile
                                )

prod_without_extreme_thresholds = product(
                                    initial_assets_list,
                                    atr_list,
                                    ebitabl_list,
                                    deductible_list,
                                    loss_ratio_list,
                                    policy_limit_list,
                                    [None],
                                    [0.0],
                                    [1.0]
                                )

prod_combined = list(prod_with_extreme_thresholds) + list(prod_without_extreme_thresholds)

total_combos = len(prod_combined)

# Parallel execution with progress bar
with TqdmJoblib(total=total_combos, desc="Parameter sets", leave=True):
    Parallel(n_jobs=n_jobs, prefer="processes", batch_size=1)(
        delayed(run_tail_simulation)(
            index, ia, atr, ebitabl, ded, lr, policy_limit,
            extreme_threshold, extreme_sev_shape, extreme_sev_scale,
            NUM_SIMULATIONS, SIM_YEARS, PRICING_SIMULATIONS,
            filepath=gdrive_results_dir
        )
        for index, (ia, atr, ebitabl, ded, lr, policy_limit,
                    extreme_threshold, extreme_sev_shape, extreme_sev_scale) in enumerate(prod_combined)
    )

Available cores: 44


Parameter sets:   0%|          | 0/208 [00:00<?, ?it/s]